# Detect Letters using the Yolo Model

In this notebook, we detect all letters in the ASC dataset we already have manual annotations for. The letters are:

* Aleph
* He
* Mem
* Shin
* Mem Sofit
* Tav


In [42]:
# Cell 1: imports & configuration

from ultralytics import YOLO
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import json
import pandas as pd
import matplotlib.pyplot as plt
import os

# ---- class & dataset configuration ----
CLASS_ID_TO_NAME = {
    0: "Aleph",
    1: "He",
    2: "Mem",
    3: "Shin",
    4: "Mem Sofit",
    5: "Tav",
}

# Script "modes"
SCRIPT_MODES = ["french", "german"]

# Paths to your YOLO config & data
MODEL_CFG   = "yolo11m.yaml"
BASE_WEIGHTS = "yolo11m.pt"        # starting weights
DATA_YAML   = "asc_dataset.yaml"   # your dataset config

# Training hyper-parameters
EPOCHS   = 300
IMGSZ    = 1500
BATCH    = 1

# Where YOLO will write runs
PROJECT = "runs/detect"

# Inference configuration
IMAGES_ROOT = Path("images")      # images/french, images/german
CONF_THRES  = 0.5
IOU_THRES   = 0.5

# Storage for later: where each class model is saved + metrics
RUN_INFO = {}

## Training

In [43]:
# Cell 2: train one model per class

for class_id, class_name in CLASS_ID_TO_NAME.items():
    run_name = f"train_cat{class_id}_{class_name.lower().replace(' ', '_')}"
    print("=" * 80)
    print(f"Training model for class {class_id}: {class_name}")
    print(f"Run name: {run_name}")
    print("=" * 80)

    # Fresh model for this class
    model = YOLO(MODEL_CFG).load(BASE_WEIGHTS)

    # Train only on this class (YOLO will internally reindex to class 0)
    _ = model.train(
        data=DATA_YAML,
        epochs=EPOCHS,
        imgsz=IMGSZ,
        batch=BATCH,
        classes=[class_id],
        project=PROJECT,
        name=run_name,
        verbose=True,
    )

    # Explicit validation for this class (using the same dataset)
    metrics = model.val(
        data=DATA_YAML,
        split="val",       # assumes val split in asc_dataset.yaml
        classes=[class_id]
    )

    # Store information for later visualization & inference
    run_dir = Path(PROJECT) / run_name
    RUN_INFO[class_id] = {
        "class_name": class_name,
        "run_name": run_name,
        "run_dir": run_dir,
        "metrics_dict": getattr(metrics, "results_dict", None),
        "metrics_obj": metrics,
    }

    # Quick textual summary for log / notebook output
    try:
        print(f"[{class_name}] mAP50-95: {metrics.box.map:.3f}")
        print(f"[{class_name}] mAP50:    {metrics.box.map50:.3f}")
        print(f"[{class_name}] mAP75:    {metrics.box.map75:.3f}")
        print(f"[{class_name}] Precision: {metrics.box.mp:.3f}")
        print(f"[{class_name}] Recall:    {metrics.box.mr:.3f}")
    except Exception as e:
        print(f"Could not access metrics fields for {class_name}: {e}")

Training model for class 0: Aleph
Run name: train_cat0_aleph
Transferred 649/649 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.229 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.62 🚀 Python-3.12.2 torch-2.5.1 CUDA:0 (NVIDIA A40, 45416MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.yaml, data=asc_dataset.yaml, epochs=300, time=None, patience=100, batch=1, imgsz=1500, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/detect, name=train_cat0_aleph3, exist_ok=False, pretrained=yolo11m.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=Fals

100%|██████████| 5.35M/5.35M [00:00<00:00, 12.2MB/s]


AMP: checks passed ✅
WARNING ⚠️ imgsz=[1500] must be multiple of max stride 32, updating to [1504]


train: Scanning /home/suliman/midrash_auto_annotate_asc/ASC_dataset2/labels/train/french.cache... 29 images, 0 backgrounds, 0 corrupt: 100%|██████████| 29/29 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/suliman/midrash_auto_annotate_asc/ASC_dataset2/images/train/german/012_000_00.jpg: 1 duplicate labels removed


...

                   all          8        250      0.895      0.925      0.941      0.805
                   Tav          8        250      0.895      0.925      0.941      0.805
Speed: 0.3ms preprocess, 10.8ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/train_cat5_tav3
Ultralytics 8.3.62 🚀 Python-3.12.2 torch-2.5.1 CUDA:0 (NVIDIA A40, 45416MiB)
YOLO11m summary (fused): 303 layers, 20,034,658 parameters, 0 gradients


val: Scanning /home/suliman/midrash_auto_annotate_asc/ASC_dataset2/labels/val/french.cache... 8 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8/8 [00:00<?, ?it/s]

val: WARNING ⚠️ /home/suliman/midrash_auto_annotate_asc/ASC_dataset2/images/val/french/016_000_00.jpg: 175 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:00<00:00, 13.95it/s]


                   all          8        250      0.899      0.925      0.941      0.809
                   Tav          8        250      0.899      0.925      0.941      0.809
Speed: 0.4ms preprocess, 18.7ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/train_cat5_tav32
[Tav] mAP50-95: 0.809
[Tav] mAP50:    0.941
[Tav] mAP75:    0.933
[Tav] Precision: 0.899
[Tav] Recall:    0.925


## Visualization

In [44]:
# Cell 3: visualize training curves and metrics for each class

def plot_training_curves_for_class(class_id, info):
    class_name = info["class_name"]
    run_dir = info["run_dir"]
    csv_path = run_dir / "results.csv"

    if not csv_path.exists():
        print(f"[{class_name}] results.csv not found at {csv_path}")
        return

    df = pd.read_csv(csv_path)

    # The exact column names depend on ultralytics version.
    # Adjust if needed by printing df.columns once.
    cols = df.columns
    print(f"[{class_name}] results.csv columns: {list(cols)}")

    # Choose some common columns (change if names differ in your file)
    # Try/except so it doesn't crash if a column is missing.
    def safe_col(name, default=None):
        return df[name] if name in df.columns else default

    epochs = df["epoch"] if "epoch" in df.columns else range(len(df))

    fig, axes = plt.subplots(1, 3, figsize=(18, 4))
    fig.suptitle(f"Training curves – {class_name} (class {class_id})", fontsize=16)

    # 1. Losses
    axes[0].plot(epochs, safe_col("train/box_loss"), label="train/box_loss")
    axes[0].plot(epochs, safe_col("val/box_loss"),   label="val/box_loss")
    axes[0].set_xlabel("Epoch")
    axes[0].set_ylabel("Loss")
    axes[0].set_title("Box loss")
    axes[0].legend()

    # 2. Precision & Recall
    p_col = "metrics/precision(B)" if "metrics/precision(B)" in cols else "metrics/precision"
    r_col = "metrics/recall(B)"    if "metrics/recall(B)" in cols    else "metrics/recall"

    axes[1].plot(epochs, safe_col(p_col), label="Precision")
    axes[1].plot(epochs, safe_col(r_col), label="Recall")
    axes[1].set_xlabel("Epoch")
    axes[1].set_ylabel("Score")
    axes[1].set_title("Precision / Recall")
    axes[1].legend()

    # 3. mAP
    map50_col    = "metrics/mAP50(B)"      if "metrics/mAP50(B)" in cols      else "metrics/mAP50"
    map5095_col  = "metrics/mAP50-95(B)"   if "metrics/mAP50-95(B)" in cols   else "metrics/mAP50-95"

    axes[2].plot(epochs, safe_col(map50_col),   label="mAP50")
    axes[2].plot(epochs, safe_col(map5095_col), label="mAP50–95")
    axes[2].set_xlabel("Epoch")
    axes[2].set_ylabel("mAP")
    axes[2].set_title("mAP over epochs")
    axes[2].legend()

    plt.tight_layout()
    plt.show()


for cid, info in RUN_INFO.items():
    plot_training_curves_for_class(cid, info)

[Aleph] results.csv columns: ['epoch', 'time', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss', 'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss', 'lr/pg0', 'lr/pg1', 'lr/pg2']


<Figure size 1800x400 with 3 Axes>

[He] results.csv columns: ['epoch', 'time', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss', 'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss', 'lr/pg0', 'lr/pg1', 'lr/pg2']


<Figure size 1800x400 with 3 Axes>

[Mem] results.csv columns: ['epoch', 'time', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss', 'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss', 'lr/pg0', 'lr/pg1', 'lr/pg2']


...

<Figure size 1800x400 with 3 Axes>

[Mem Sofit] results.csv columns: ['epoch', 'time', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss', 'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss', 'lr/pg0', 'lr/pg1', 'lr/pg2']


<Figure size 1800x400 with 3 Axes>

[Tav] results.csv columns: ['epoch', 'time', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss', 'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss', 'lr/pg0', 'lr/pg1', 'lr/pg2']


<Figure size 1800x400 with 3 Axes>

In [45]:
# Optional: a compact table of final metrics per class

rows = []
for cid, info in RUN_INFO.items():
    m = info["metrics_obj"]
    try:
        rows.append({
            "class_id": cid,
            "class_name": info["class_name"],
            "mAP50-95": float(m.box.map),
            "mAP50":    float(m.box.map50),
            "mAP75":    float(m.box.map75),
            "Precision": float(m.box.mp),
            "Recall":    float(m.box.mr),
        })
    except Exception as e:
        print(f"Skipping metrics for {info['class_name']}: {e}")

metrics_df = pd.DataFrame(rows)
display(metrics_df)

,class_id,class_name,mAP50-95,mAP50,mAP75,Precision,Recall
0,0,Aleph,0.873847,0.988994,0.972535,0.969921,0.972621
1,1,He,0.800005,0.942921,0.925799,0.914912,0.963054
2,2,Mem,0.828775,0.976980,0.963737,0.973772,0.969723
3,3,Shin,0.905801,0.993527,0.987961,0.995984,0.968749
4,4,Mem Sofit,0.839860,0.970817,0.969961,0.906650,0.940789
5,5,Tav,0.809301,0.940636,0.933371,0.898973,0.925435


## Inference

In [52]:
# Cell 4: helper to run YOLO on a directory and export COCO JSON for ONE class

def export_coco_predictions_for_class_and_script(
    class_id: int,
    class_name: str,
    weights_path: Path,
    script_mode: str,
    images_root: Path = IMAGES_ROOT,
    conf_thres: float = CONF_THRES,
    iou_thres: float = IOU_THRES,
    imgsz: int = IMGSZ,
):
    """
    Run a trained single-class YOLO model on all images under images_root/script_mode,
    and export predictions as a COCO JSON: yolo_preds_cat{class_id}_{script_mode}.json
    """

    # 1) Load model
    print(f"\nLoading model for {class_name} from {weights_path}")
    model = YOLO(str(weights_path))

    # 2) Collect images
    script_dir = images_root / script_mode
    # Handle possible subdirectories (if any)
    image_paths = sorted(
        [p for p in script_dir.rglob("*") if p.suffix.lower() in [".jpg", ".jpeg", ".png", ".tif", ".tiff"]]
    )
    print(f"Found {len(image_paths)} images under {script_dir}")

    images = []
    annotations = []
    categories = [
        {
            "id": class_id + 1, 
            "name": class_name,
            "supercategory": "letter",
        }
    ]

    ann_id = 1

    for img_path in tqdm(image_paths, desc=f"{class_name} – {script_mode}"):
        # Extract image_id from filename: x_y_z.ext → 100*x + 10*y + z
        name = img_path.stem  # e.g., "012_034_07"
        parts = name.split("_")

        if len(parts) != 3:
            print(f"Skipping {img_path}: unexpected filename format.")
            continue

        try:
            x = int(parts[0])  # 3-digit
            y = int(parts[1])  # 3-digit
            z = int(parts[2])  # 2-digit
        except ValueError:
            print(f"Skipping {img_path}: cannot parse x_y_z structure.")
            continue

        image_id = 100 * x + 10 * y + z
        # Read image size
        try:
            with Image.open(img_path) as im:
                width, height = im.size
        except Exception as e:
            print(f"Skipping {img_path} due to error: {e}")
            continue

        images.append(
            {
                "id": image_id,
                "file_name": str(img_path.name),
                "width": width,
                "height": height,
            }
        )

        # 3) Run inference
        results = model.predict(
            source=str(img_path),
            conf=conf_thres,
            iou=iou_thres,
            imgsz=imgsz,
            verbose=False,
        )

        if not results:
            continue

        r = results[0]
        if r.boxes is None or len(r.boxes) == 0:
            continue

        # YOLO was trained as a single-class detector, so we ignore the predicted class
        for box in r.boxes:
            xyxy = box.xyxy[0].tolist()  # [x1, y1, x2, y2]
            x1, y1, x2, y2 = xyxy
            w = x2 - x1
            h = y2 - y1

            if w <= 0 or h <= 0:
                continue

            annotations.append(
                {
                    "id": ann_id,
                    "image_id": image_id,
                    "category_id": class_id + 1,
                    "bbox": [int(x1), int(y1), int(w), int(h)],
                    "area": int(w * h),
                    "iscrowd": 0,
                }
            )
            ann_id += 1

    OUTPUT_PATH = "yolo_preds"
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    out_json = f"{OUTPUT_PATH}/yolo_preds_cat{class_id + 1}_{script_mode}.json"
    coco_dict = {
        "images": images,
        "annotations": annotations,
        "categories": categories,
    }

    with open(out_json, "w", encoding="utf-8") as f:
        json.dump(coco_dict, f, indent=4)  # <-- indented output as requested

    print(
        f"Saved {len(annotations)} predictions for {len(images)} images "
        f"({script_mode}, class {class_id} – {class_name}) ➜ {out_json}"
    )

In [53]:
# Cell 5: generate COCO predictions for ALL classes and BOTH script modes

for class_id, info in RUN_INFO.items():
    class_name = info["class_name"]
    run_name   = info["run_name"]
    run_dir    = info["run_dir"]

    weights_path = run_dir / "weights" / "best.pt"
    if not weights_path.exists():
        print(f"WARNING: weights not found for {class_name} at {weights_path}, skipping.")
        continue

    for script_mode in SCRIPT_MODES:
        export_coco_predictions_for_class_and_script(
            class_id=class_id,
            class_name=class_name,
            weights_path=weights_path,
            script_mode=script_mode,
        )


Loading model for Aleph from runs/detect/train_cat0_aleph/weights/best.pt
Found 144 images under images/french


Aleph – french:   0%|          | 0/144 [00:00<?, ?it/s]

WARNING ⚠️ imgsz=[1500] must be multiple of max stride 32, updating to [1504]


Aleph – french:   1%|          | 1/144 [00:00<00:54,  2.63it/s]

WARNING ⚠️ imgsz=[1500] must be multiple of max stride 32, updating to [1504]
WARNING ⚠️ imgsz=[1500] must be multiple of max stride 32, updating to [1504]


...

WARNING ⚠️ imgsz=[1500] must be multiple of max stride 32, updating to [1504]
WARNING ⚠️ imgsz=[1500] must be multiple of max stride 32, updating to [1504]


Tav – german:  99%|█████████▊| 154/156 [00:22<00:00, 11.98it/s]

WARNING ⚠️ imgsz=[1500] must be multiple of max stride 32, updating to [1504]
WARNING ⚠️ imgsz=[1500] must be multiple of max stride 32, updating to [1504]


Tav – german: 100%|██████████| 156/156 [00:22<00:00,  6.81it/s]

Saved 3914 predictions for 156 images (german, class 5 – Tav) ➜ yolo_preds/yolo_preds_cat6_german.json
